# Testing Code


Before submiting the final answers, this jupyter notebook could be used to experiment and verify if the code is working!


Here we are taking inspiration from two of the top projects:
 * https://huggingface.co/spaces/baixianger/RobotPai
 * https://huggingface.co/spaces/fisherman611/gaia-agent/

In [ ]:
%load_ext autoreload
%autoreload 2

## Load Data

In [1]:
import json

# Load the metadata.jsonl file
with open('data/metadata.jsonl', 'r') as jsonl_file:
    json_list = list(jsonl_file)

json_QA = []
for json_str in json_list:
    json_data = json.loads(json_str)
    json_QA.append(json_data)

### Data Analyses

Here we analyze the data in order to find what tools we need to build a robust agent

In [ ]:
import random

random_samples = random.sample(json_QA, 3)
for sample in random_samples:
    print("=" * 50)
    print(f"Task ID: {sample['task_id']}")
    print(f"Question: {sample['Question']}")
    print(f"Level: {sample['Level']}")
    print(f"Final Answer: {sample['Final answer']}")
    print(f"Annotator Metadata: ")
    print(f"  ├── Steps: ")
    for step in sample['Annotator Metadata']['Steps'].split('\n'):
        print(f"  │      ├── {step}")
    print(f"  ├── Number of steps: {sample['Annotator Metadata']['Number of steps']}")
    print(f"  ├── How long did this take?: {sample['Annotator Metadata']['How long did this take?']}")
    print(f"  ├── Tools:")
    for tool in sample['Annotator Metadata']['Tools'].split('\n'):
        print(f"  │      ├── {tool}")
    print(f"  └── Number of tools: {sample['Annotator Metadata']['Number of tools']}")
print("=" * 50)

In [3]:
### Linking to supabase server
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client


load_dotenv()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2") #  dim=768

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

### If you already have embeddings in your db, skip the next cell 👇

In [ ]:
# Questions to list of documents
from langchain.schema import Document
import uuid

docs = []
for sample in json_QA:
    content = f"Question : {sample['Question']}\n\nFinal answer : {sample['Final answer']}"
    doc = {
        "id": str(uuid.uuid4()),  # 手动生成 UUID
        "content" : content,
        "metadata" : { 
            "source" : sample['task_id']
        },
        "embedding" : embeddings.embed_query(content),
    }
    docs.append(doc)

print('docs')
print(docs[0])
# upload the documents to the vector database
try:
    response = supabase.table("documents").insert(docs).execute()
    print("Data inserted successfully:", response)
except Exception as e:
    print("Error inserting data into Supabase:", e)

In [4]:
# add items to vector database
vector_store = SupabaseVectorStore(
    client=supabase,
    embedding= embeddings,
    table_name="documents",
    query_name="match_documents",
)
retriever = vector_store.as_retriever()

In [ ]:
query = "On June 6, 2023, an article by Carolyn Collins Petersen was published in Universe Today. This article mentions a team that produced a paper about their observations, linked at the bottom of the article. Find this paper. Under what NASA award number was the work performed by R. G. Arendt supported by?"
# matched_docs = vector_store.similarity_search(query, 2)
docs = retriever.invoke(query)
docs[0]

In [ ]:
from collections import Counter, OrderedDict

tools = []
for sample in json_QA:
    for tool in sample['Annotator Metadata']['Tools'].split('\n'):
        tool = tool[2:].strip().lower()
        if tool.startswith("("):
            tool = tool[11:].strip()
        tools.append(tool)
tools_counter = OrderedDict(Counter(tools))
print("List of tools used in all samples:")
print("Total number of tools used:", len(tools_counter))
for tool, count in tools_counter.items():
    print(f"  ├── {tool}: {count}")

## Graph Implementation

Here we build the agent graph with LangGraph to solve those tasks!

In [ ]:
system_prompt = """
You are a helpful assistant tasked with answering questions using a set of tools.
If the tool is not available, you can try to find the information online. You can also use your own knowledge to answer the question. 
You need to provide a step-by-step explanation of how you arrived at the answer.
==========================
Here is a few examples showing you how to answer the question step by step.
"""
for i, samples in enumerate(random_samples):
    system_prompt += f"\nQuestion {i+1}: {samples['Question']}\nSteps:\n{samples['Annotator Metadata']['Steps']}\nTools:\n{samples['Annotator Metadata']['Tools']}\nFinal Answer: {samples['Final answer']}\n"
system_prompt += "\n==========================\n"
system_prompt += "Now, I will ask you a question. Please answer the question step by step. Report your thoughts, and finish your answer with the following template: \n"
system_prompt += "FINAL ANSWER: [YOUR FINAL ANSWER]. \n"
system_prompt += "YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. If you are asked for a comma separated list, Apply the rules above for each element (number or string), ensure there is exactly one space after each comma.\n"
system_prompt += "Your answer should only start with 'FINAL ANSWER: ', then follows with the answer.\n"

# save the system_prompt to a file
with open('system_prompt.txt', 'w') as f:
    f.write(system_prompt)

In [ ]:
# load the system prompt from the file
with open('system_prompt.txt', 'r') as f:
    system_prompt = f.read()
print(system_prompt)

In [2]:
import os
from dotenv import load_dotenv
from langgraph.graph import MessagesState, START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from tools.searchtools import question_retrieve_tool, wiki_search, web_search, arxiv_search
from tools.mathtools import multiply, add, subtract, divide, modulus
from tools.documenttools import download_file_from_url, save_file, extract_text_from_image, analyze_csv_file, analyze_excel_file, construct_question_related_file_url
from tools.audiotools import transcribe_audio
from tools.codetools import execute_code_multilang

# Define the retriever from supabase
load_dotenv()



tools = [
    multiply,
    add,
    subtract,
    divide,
    modulus,
    wiki_search,
    web_search,
    arxiv_search,
    question_retrieve_tool,
    construct_question_related_file_url,
    download_file_from_url,
    save_file,
    extract_text_from_image,
    analyze_csv_file,
    analyze_excel_file,
    execute_code_multilang,
    transcribe_audio,
]

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url='https://openrouter.ai/api/v1'
)
llm_with_tools = llm.bind_tools(tools)

In [3]:
# load the system prompt from the file
with open('system_prompt.txt', 'r') as f:
    system_prompt = f.read()


# System message
sys_msg = SystemMessage(content=system_prompt)

# Node
def assistant(state: MessagesState):
    """Assistant node"""
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

# Compile graph
graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [13]:
import requests

random_question = requests.get(
    "https://agents-course-unit4-scoring.hf.space/random-question").json()

question = random_question["question"]
question_id = random_question["task_id"]
file_name = random_question["file_name"]

print(question)
print(question_id)
print(file_name)

messages = [HumanMessage(content=question)]
if file_name:
    messages.append(HumanMessage(content=f"Question id is: {question_id}"))

messages = graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()

How many at bats did the Yankee with the most walks in the 1977 regular season have that same season?
3f57289b-8c60-48be-bd80-01f8099ca449

================================ Human Message =================================

How many at bats did the Yankee with the most walks in the 1977 regular season have that same season?
================================== Ai Message ==================================

To find out how many at bats the Yankee with the most walks in the 1977 regular season had, we need to follow these steps:

1. Identify the Yankee player with the most walks in the 1977 regular season.
2. Determine the number of at bats that player had in the same season.

Let's start by identifying the player with the most walks. Reggie Jackson led the New York Yankees in walks during the 1977 regular season with 86 walks.

Next, we need to find out how many at bats Reggie Jackson had in the 1977 regular season. Reggie Jackson had 582 at bats in the 1977 regular season.

Therefore, the 

In [14]:
question = "How many at bats did the Yankee with the most walks in the 1977 regular season have that same season?"
question_id = "Question id is: 3f57289b-8c60-48be-bd80-01f8099ca449"
messages = [HumanMessage(content=question), HumanMessage(content=question_id)]
messages = graph.invoke({"messages": messages})

In [15]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

How many at bats did the Yankee with the most walks in the 1977 regular season have that same season?
================================ Human Message =================================

Question id is: 3f57289b-8c60-48be-bd80-01f8099ca449
================================== Ai Message ==================================
Tool Calls:
  construct_question_related_file_url (call_yLXQpfOlyHSeSgownKMdQwW8)
 Call ID: call_yLXQpfOlyHSeSgownKMdQwW8
  Args:
    question_id: 3f57289b-8c60-48be-bd80-01f8099ca449
================================= Tool Message =================================
Name: construct_question_related_file_url

The URL of the file related to the question is https://agents-course-unit4-scoring.hf.space/files/3f57289b-8c60-48be-bd80-01f8099ca449
================================== Ai Message ==================================
Tool Calls:
  download_file_from_url (call_yi5nEA78AIvDiyo7r7JZ8JoB)
 Call I

cornstarch, freshly squeezed lemon juice, granulated sugar, pure vanilla extract, ripe strawberries